# Working with MPAS Grids

Authors: [Philip Chmielowiec](https://github.com/philipc2), [Ian Franda](https://github.com/ifranda)



This notebook showcases how to work with datasets from the Model for Prediction Across Scales (MPAS).


##  MPAS Grid Overview

The defining feature of MPAS when compared to other models is that its unstructured grid is composed of Voronoi Meshes with a C-grid staggering. This means that the grid can be broken down into two meshes: Primal and Dual. The Primal Mesh is composed of Voronoi regions and the Dual Mesh is composed of Delaunay Triangles. The figure below showcases this relationship, with the dashed triangles being the dual of the Voronoi mesh.

<p align="center">
  <img src="../_static/examples/mpas/c-grid.png"
  width="400" / >
</p>

Since the Primal Mesh is predominantly made up of hexagons, with pentagons and heptagons occasionally being present, and the Dual Mesh being strictly triangular, this notebook will showcase how we can represent both of these meshes in the **UGRID** conventions using **UXarray**.





## Dataset Overview

Before diving straight into using **UXarray**, it is important to first investigate how MPAS datasets are represented.

As mentioned in earlier notebooks, the grid definition and data variables are typically stored in separate files. However, in this example, our dataset will contain both within the same file, which is often the case when working with smaller datasets. Additionally, even when working with separate Grid and Data files in MPAS, the definition of the Primal and Dual mesh are still stored under the same Grid file.

Below we can take a quick look into the dataset by opening it with **Xarray**.


In [ ]:
# Dataset Path
mpas_root_filepath = "../../test/meshfiles/mpas/"
mpas_dataset_filepath = mpas_root_filepath + "QU/mesh.QU.1920km.151026.nc"

In [ ]:
import xarray as xr

xrds_mpas = xr.open_dataset(mpas_dataset_filepath)
xrds_mpas

Here we opened up the dataset to get an overview of the full set of grid variables needed to describe an MPAS grid as outlined in the MPAS Specification Document [2]. Below is a list of the key grid variables that are used for representing and constructing the Primal and Dual meshes.


### Primal Mesh
* **lonVertex, latVertex**: Corner Vertices of Primal Mesh cells
* **lonCell, latCell**: Center Vertices of Primal Mesh cells
* **verticesOnCell**: Vertex indices that surround each Primal Mesh cell
* **verticesOnEdge**: Vertex indices that saddle a given edge
* **nEdgesOnCell**: Maximum number of edges that can surround a cell

### Dual Mesh
* **lonCell, latCell**: Corner Vertices of Dual Mesh cells
* **lonVertex, latVertex**: Center Vertices of Dual Mesh cells
* **cellsOnVertex**: Vertex indices that surround each Dual Mesh cell
* **cellsOnEdge**: Vertex indices that saddle a given edge


## Constructing a Grid Object

```{note}
Since we *only* have a Grid file and no Data file in this example, we will be working exclusively with the `Grid` class to investigate the grid topology and not with `UxDataset` or `UxDataArray` data structures.
```

The `xarray.Dataset` that we opened in the previous section stores the coordinates and connectivity variables according to the MPAS specification standards for both the Primal and Dual meshes together in a single dataset. Here, instead of opening up the dataset using **Xarray**, we can pass through the path into our `open_grid` method to construct an instance of a `Grid` class. This `Grid` can take in a `use_dual` parameter to select whether to construct the Primal or Dual mesh, parsing and encoding the appropriate variables in the UGRID conventions.

In [ ]:
import uxarray as ux

primal_mesh = ux.open_grid(mpas_dataset_filepath, use_dual=False)
dual_mesh = ux.open_grid(mpas_dataset_filepath, use_dual=True)

In [ ]:
primal_mesh

In [ ]:
dual_mesh

## Relationship between MPAS and UGRID

In the previous two sections, we outlined the set of grid variables used to describe the Primal and Dual meshes and how to open an MPAS grid in **UXarray**. Here, we provide an overview of how we represent both meshes in the UGRID conventions and how the original grid variables were modified to meet these conventions.

### Grid Variables (Primal Mesh)
`Mesh2_node_x` & `Mesh2_node_y`
* Longitude and Latitude coordinates of the Primal Mesh corner nodes
* Derived from `lonVertex` & `latVertex`
* Converted from Radians to Degrees

`Mesh2_face_x` & `Mesh2_face_y`
* Longitude and Latitude coordinates of the Primal Mesh center nodes
* Derived from `lonCell` & `latCell`
* Converted from Radians to Degrees

`Mesh2_face_nodes`
* Connectivity array describing which nodes make up a face
* Derived from `verticesOnCell`
* Padding is replaced with `INT_FILL_VALUE`
* Missing Values (zeros) replaced with `INT_FILL_VALUE`
* Converted to zero-index

`Mesh2_edge_nodes`
* Connectivity array describing which nodes link to form each edge
* Derived from `verticesOnEdge`
* Padding is replaced with `INT_FILL_VALUE`
* Missing Values (zeros) replaced with `INT_FILL_VALUE`
* Converted to zero-index

### Grid Variables (Dual Mesh)

`Mesh2_node_x` & `Mesh2_node_y`
* Longitude and Latitude coordinates of the Dual Mesh vertices
* Derived from `lonCell` & `latCell`, the centers of the Primal Mesh
* Converted from Radians to Degrees

`Mesh2_face_x` & `Mesh2_face_y`
* Longitude and Latitude coordinates of the Dual Mesh centers
* Derived from `lonVertex` & `latVertex`, the vertices of the Primal Mesh
* Converted from Radians to Degrees

`Mesh2_face_nodes`
* Connectivity array describing which nodes make up a face
* Derived from `verticesOnCell`
* Padding is replaced with `INT_FILL_VALUE`
* Missing Values (zeros) replaced with `INT_FILL_VALUE`
* Converted to zero-index

`Mesh2_edge_nodes`
* Connectivity array describing which nodes link to form each edge
* Derived from `verticesOnEdge`
* Padding is replaced with `INT_FILL_VALUE`
* Missing Values (zeros) replaced with `INT_FILL_VALUE`
* Converted to zero-index

## Functionality

### Face Area Calculation

Using our parsed attributes, we can determine whether our unstructured grid lies on the surface of a sphere by accessing the `on_a_sphere` attribute.


In [ ]:
primal_mesh.parsed_attrs['on_a_sphere']

Simiarly, we can access the `sphere_radius` attribute.

In [ ]:
primal_mesh.parsed_attrs['sphere_radius']

Since our mesh lies on a sphere, we would expect our total surface area to follow the equation

{math}`4{\pi}{r^2}`

We can use the value of the `sphere_radius` attribute to calculate the expected total surface area.

In [ ]:
import numpy as np

sphere_r = primal_mesh.parsed_attrs['sphere_radius']
expected_area = 4 * np.pi * (sphere_r)**2
expected_area

`UXarray` can be used to compute the face area of each face on our grid.

In [ ]:
primal_mesh_face_areas = primal_mesh.face_areas
primal_mesh_face_areas

The total face (surface) area can be computed by summing over each value.

In [ ]:
primal_mesh_face_areas.sum()

We can then compute the absolute error of our calculation.

In [ ]:
abs(expected_area - primal_mesh_face_areas.sum())

The same can be done for the Dual Mesh.

In [ ]:
dual_mesh_face_areas = dual_mesh.face_areas
dual_mesh_face_areas.sum()

In [ ]:
abs(expected_area - dual_mesh_face_areas.sum())

We can see that the total face area of both the Primal and Dual meshes is within 1e-6 of the expected area. For a more detailed explanation of the face area calculation and ways to obtain more precision, check out our other notebooks.

## Visualization

To visually confirm that the Primal and Dual meshes have been correctly translated to the **UGRID** conventions, the following showcases a basic visualization of the mesh structure.

To display the mesh, the vertices are used to construct a `PolyCollection` object, which represents a group of polygons on a plane. The collection is then fed into Matplotlib to be visualized.


```{note}
The following visualizations showcase the mesh structure excluding any faces that are located on the boundary between 0 and 360 degrees. There is work currently being done to handle these cases and to provide a more sophisticated visualization API in UXarray. Stay tuned!
```

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.collections import PolyCollection

from uxarray.grid.connectivity import close_face_nodes

### Helper Functions

In order to make our data compatible wth the `PolyCollection` class, we need to first convert it into an appropriate representation. The following helper function takes in a `Grid` object (either Primal or Dual mesh) and returns the vertices of each face that can be interpreted by the `PolyCollection`.

In [ ]:
def get_verts(grid):
    nodes_per_face = grid.nNodes_per_face.values
    x = grid.Mesh2_node_x.values
    y = grid.Mesh2_node_y.values
    closed_face_nodes = close_face_nodes(grid.Mesh2_face_nodes.values,
                                         grid.nMesh2_face,
                                         grid.nMaxMesh2_face_nodes)

    verts = []
    for face_node, n_nodes in zip(closed_face_nodes, nodes_per_face):
        polygon_x = x[face_node[0:n_nodes]]
        polygon_y = y[face_node[0:n_nodes]]

        # exclude faces near the boundary
        if np.any(np.abs(polygon_x - 180) > 170):
            continue

        vert = np.array([polygon_x, polygon_y])
        verts.append(vert.T)

    return verts

### Primal Mesh

The following displays the Primal Mesh, which is composed mostly of hexagons representing the Voronoi regions,


In [ ]:
primal_vertices = get_verts(primal_mesh)
primal_polygons = PolyCollection(primal_vertices, edgecolors='Black', alpha=0.5)

fig, ax = plt.subplots()
ax.add_collection(primal_polygons, autolim=False)
ax.set_xlim(0, 360)
ax.set_ylim(-80, 80)
ax.set_title("Primal Mesh")

### Dual Mesh

The follwing displays the Dual Mesh, which is composed entirely of Delaunay triangles.

In [ ]:
dual_vertices = get_verts(dual_mesh)
dual_polygons = PolyCollection(dual_vertices, edgecolors='Black', alpha=0.5)

fig, ax = plt.subplots()
ax.add_collection(dual_polygons, autolim=False)
ax.set_xlim(0, 360)
ax.set_ylim(-80, 80)
ax.set_title("Dual Mesh")

## Primal and Dual Meshes Overlayed

As mentioned earlier, the Primal and Dual meshes are related to one another. The vertices of the Primal mesh are each at the center of a Delaunay triangle in the Dual mesh. Likewise, the vertices of the Dual mesh are each at the center of a Voronoi region in the Primal mesh. This relationship can be seen in the visualization below

In [ ]:
dual_vertices = get_verts(dual_mesh)
dual_polygons = PolyCollection(dual_vertices, edgecolors='Black', alpha=0.5)
primal_vertices = get_verts(primal_mesh)
primal_polygons = PolyCollection(primal_vertices, edgecolors='Black', alpha=0.5)

fig, ax = plt.subplots()
ax.add_collection(primal_polygons, autolim=False)
ax.add_collection(dual_polygons, autolim=False)
ax.set_xlim(0, 360)
ax.set_ylim(-80, 80)
ax.set_title("Dual & Primal Mesh")

## References

[1] [https://mpas-dev.github.io/](https://mpas-dev.github.io/)

[2] [https://mpas-dev.github.io/files/documents/MPAS-MeshSpec.pdf](https://mpas-dev.github.io/files/documents/MPAS-MeshSpec.pdf)

[3] [http://ugrid-conventions.github.io/ugrid-conventions/](http://ugrid-conventions.github.io/ugrid-conventions/)